In [2]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [3]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [19]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    # IMP : initialize the value function to zeros initially, check if the results change with different initialilzations - shouldn't change ideally

    V = np.zeros(env.nS)
    # the V array is not changed in the following iterations, this is imp! :P
   
    while True:
    # TODO: Implement!
        threshold = 0

        for s in range(env.nS):
            v = 0

            # here a is just an index to the set of all possible actions from state s, see - http://book.pythontips.com/en/latest/enumerate.html
            # we are going over all the possible actions from state s
            # policy matrix contains the probability of an action being taken from a state
            for a, action_prob in enumerate(policy[s]):
                # env.P[s][a] has the transition probabilities as given by the environment - here we are using the environment's knowledge in making decisions
                for prob, next_state, reward, info in env.P[s][a]:
                    # the order of these values is important, don't change the order! - needs lots of debugging
                    v += action_prob * prob *(reward + discount_factor * V[next_state])
                    # bootstrapping - guessing V[next_state] and using it to make current calculation

            threshold = max(threshold, v - V[s])
            V[s] = v
        # stop if the maximum change in value function is less than some threshold
        if threshold < theta:
            break
    # why return np.array(V)? V was an numpy nd-array anyways? The output seems fine though     
    return V

In [20]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # we need a helper function to return the expected values that can be obtained if we follow different actions from
    # a state
    def one_step_lookahead(state, V, env):
        """
        Args:
        state: The state under consideration
        V: The value function for all the states
        env: The gridworld environment
        
        Returns:
        An array of env.nA size, containing the values that can be obtained from all the possible actions
        """
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, info in env.P[state][a]:
                A[a] += prob * (reward + V[next_state])
                
        return A
            
            
    
    
    
    
    
    while True:
        # Implement this!
        v = policy_eval(policy, env)
        stable = True # variable to know if we have selected an optimal policy
        for s in range(env.nS):
            old_choice = np.argmax(policy[s])
            
            action_values = one_step_lookahead(s, v, env)
            current_choice = np.argmax(action_values)
            
            if current_choice != old_choice:
                stable = False # we have not yet reached the optimal policy
            
            # updating the policy to choose the greedy option
            policy[s] = np.eye(env.nA)[current_choice]
            # the first part creates an identity matrix of env.nA dimensions.
            # the second part ensures only the current_choice row is selected from this row
            # so essentially, this is like one-hot encoding the best choice 
            
            
        if stable == True:
            break
    
    return policy, np.zeros(env.nS)

In [21]:
policy = np.ones([env.nS, env.nA]) / env.nA
# Start with a random policy

for _ in range(100):
    policy_eval(policy, env)
    policy_improvement(env)

In [23]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



Policy Probability Distribution:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 0 0]
 [0 0 0 0]
 [0 1 1 2]
 [2 0 1 0]]

Value Function:
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

Reshaped Grid Value Function:
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]



In [24]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

AssertionError: 
Arrays are not almost equal to 2 decimals

(mismatch 87.5%)
 x: array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.])
 y: array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])